In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("Name:", gpu.name, "  Type:", gpu.device_type)

    except RuntimeError as e:
        print(e)

Name: /physical_device:GPU:0   Type: GPU


In [2]:
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt
from sklearn.utils import shuffle as skshuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
# from tensorflow.python.keras.layers.recurrent import LSTM
from pylab import rcParams
%matplotlib inline

In [3]:
np.__version__

'1.18.5'

In [4]:
tf.__version__

'2.4.1'

In [5]:
rcParams['figure.figsize'] = 50, 10

In [6]:
PATH_TO_TEST_TRAIN_DATA = 'Tangni/Data/'

In [7]:
class LSTM_Tangni:
    def __init__(self,dictionary):
        
        self.train_size = 62
        self.data1 = pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '1-3.csv',  header=0, usecols=[0, 1, 2, 3], index_col=None)
        self.data2 = pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '2-12.csv', header=0, usecols=[0, 1, 2, 3], index_col=None)
        self.data3 = pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '3-6.csv',  header=0, usecols=[0, 1, 2, 3], index_col=None)
        self.data4 = pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '4-15.csv', header=0, usecols=[0, 1, 2, 3], index_col=None)
        self.data5 = pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '5-15.csv', header=0, usecols=[0, 1, 2, 3], index_col=None)
        
    def Print(self):
        print(self.data1)
        
    def rmse(self, true, predict):
        
        r = true - predict
        sq = r*r
        avg = np.mean(sq)
        error = math.sqrt(avg)

        return error
    
    def Average(self, lst):
        return sum(lst) / len(lst)
    
    def dataProcessing(self, dF, lookback):
    
        data=list()
        label=list()
        for i in range(len(dF) - lookback):

            data.append(np.array(dF.iloc[i:i+lookback,:4]))

            label.append(np.array(dF.iloc[i+lookback,1:2]))
        
        return np.array(data), np.array(label)

    def make_packets(self, lookback):
        
        X1, L1 = self.dataProcessing(self.data1, lookback)
        X2, L2 = self.dataProcessing(self.data2, lookback)
        X3, L3 = self.dataProcessing(self.data3, lookback)
        X4, L4 = self.dataProcessing(self.data4, lookback)
        X5, L5 = self.dataProcessing(self.data5, lookback)
        
        train_X1, label_X1,test_X1,tst_label_X1 = X1[:self.train_size - lookback], L1[:self.train_size - lookback], X1[self.train_size - lookback:], L1[self.train_size - lookback:]
        train_X2, label_X2,test_X2,tst_label_X2 = X2[:self.train_size - lookback], L2[:self.train_size - lookback], X2[self.train_size - lookback:], L2[self.train_size - lookback:]
        train_X3, label_X3,test_X3,tst_label_X3 = X3[:self.train_size - lookback], L3[:self.train_size - lookback], X3[self.train_size - lookback:], L3[self.train_size - lookback:]
        train_X4, label_X4,test_X4,tst_label_X4 = X4[:self.train_size - lookback], L4[:self.train_size - lookback], X4[self.train_size - lookback:], L4[self.train_size - lookback:]
        train_X5, label_X5,test_X5,tst_label_X5 = X5[:self.train_size - lookback], L5[:self.train_size - lookback], X5[self.train_size - lookback:], L5[self.train_size - lookback:]
        
        return train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5

    def LSTM_model(self, units, lookback):
        
        model = Sequential()
        model.add(LSTM(units = units, return_sequences = False, input_shape = (lookback,4)))

        model.add(Dense(units = 1, activation= 'linear'))
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')
#         model.summary()
        
        return model
    
    def runModel(self, units, lookback = 1, batch_size = 4, shuffle = 0):
        
        train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5 = self.make_packets(lookback)
        DATA = [train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5]
        
        Train = np.append(train_X1, np.append(train_X2, np.append(train_X3, np.append(train_X4, train_X5, axis=0), axis=0), axis=0), axis=0)
        Label = np.append(label_X1, np.append(label_X2, np.append(label_X3, np.append(label_X4, label_X5, axis=0), axis=0), axis=0), axis=0) 
        
        if (shuffle == 1):
            Train, Label = skshuffle(Train, Label, random_state=0)
            
        LSTM = self.LSTM_model(units, lookback)
        LSTM.fit(Train, Label, epochs = 10, batch_size = batch_size, verbose=0)
        
        RMSE = list()
        
        #Calculate the error of the traing and testing
        for i in range(0, 20, 2):
            
            prediction = LSTM.predict(DATA[i])
            RMSE.append(round(self.rmse(prediction, DATA[i+1]), 3))
         
        #Find average of the training and testing
        RMSE.append(round(self.Average(RMSE[0:5]), 3))
        RMSE.append(round(self.Average(RMSE[5:10]), 3))
        
        
        return RMSE
            

In [8]:
dictionary = {}

In [9]:
lstm = LSTM_Tangni(dictionary)

In [10]:
lookback = [1,2,3,4,5]
batch_size = [4,8,16,32,64]
shuffle = [0,1]
units = [10, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [11]:
Parameters = []
Performance =[]
minimum = 100
idx = 0
minidx = 0

for i in lookback:
    for j in batch_size:
        for k in shuffle:
            for u in units:
            
                Parameters.append([u,i,j,k])
                per = lstm.runModel(units = u, lookback = i, batch_size = j, shuffle = k)
                Performance.append(per)
                
                print([u,i,j,k],per[-2:])
                
                if per[-2] < minimum:
                    minimum = per[-2]
                    minidx = idx
                    
                idx += 1
                
print('--------------------------------------------------')
print('Minimimu Error at')
print(Parameters[minidx])
print(Performance[minidx])

[10, 1, 4, 0] [0.886, 0.954]
[50, 1, 4, 0] [0.345, 0.704]
[100, 1, 4, 0] [0.362, 0.73]
[150, 1, 4, 0] [0.3, 0.684]
[200, 1, 4, 0] [0.349, 0.734]
[250, 1, 4, 0] [0.421, 0.798]
[300, 1, 4, 0] [0.353, 0.731]
[350, 1, 4, 0] [0.306, 0.626]
[400, 1, 4, 0] [0.378, 0.805]
[450, 1, 4, 0] [0.338, 0.654]
[500, 1, 4, 0] [0.313, 0.69]
[10, 1, 4, 1] [0.977, 0.804]
[50, 1, 4, 1] [0.36, 0.728]
[100, 1, 4, 1] [0.57, 0.952]
[150, 1, 4, 1] [0.358, 0.721]
[200, 1, 4, 1] [0.315, 0.705]
[250, 1, 4, 1] [0.381, 0.771]
[300, 1, 4, 1] [0.335, 0.669]
[350, 1, 4, 1] [0.345, 0.67]
[400, 1, 4, 1] [0.469, 0.878]
[450, 1, 4, 1] [0.519, 0.913]
[500, 1, 4, 1] [0.334, 0.695]
[10, 1, 8, 0] [1.642, 1.404]
[50, 1, 8, 0] [0.43, 0.9]
[100, 1, 8, 0] [0.406, 0.786]
[150, 1, 8, 0] [0.352, 0.701]
[200, 1, 8, 0] [0.362, 0.806]
[250, 1, 8, 0] [0.337, 0.728]
[300, 1, 8, 0] [0.451, 0.858]
[350, 1, 8, 0] [0.348, 0.71]
[400, 1, 8, 0] [0.276, 0.671]
[450, 1, 8, 0] [0.468, 0.842]
[500, 1, 8, 0] [0.355, 0.666]
[10, 1, 8, 1] [1.321, 1.063

[400, 3, 16, 0] [0.665, 1.127]
[450, 3, 16, 0] [0.339, 0.857]
[500, 3, 16, 0] [0.566, 0.735]
[10, 3, 16, 1] [1.676, 1.292]
[50, 3, 16, 1] [0.578, 0.874]
[100, 3, 16, 1] [0.406, 0.862]
[150, 3, 16, 1] [0.39, 0.774]
[200, 3, 16, 1] [0.376, 0.831]
[250, 3, 16, 1] [0.456, 0.881]
[300, 3, 16, 1] [0.424, 0.82]
[350, 3, 16, 1] [0.453, 0.949]
[400, 3, 16, 1] [0.373, 0.704]
[450, 3, 16, 1] [0.378, 0.908]
[500, 3, 16, 1] [0.678, 1.131]
[10, 3, 32, 0] [2.757, 2.131]
[50, 3, 32, 0] [0.848, 0.87]
[100, 3, 32, 0] [0.504, 0.828]
[150, 3, 32, 0] [0.462, 0.925]
[200, 3, 32, 0] [0.425, 0.838]
[250, 3, 32, 0] [0.433, 0.819]
[300, 3, 32, 0] [0.458, 0.951]
[350, 3, 32, 0] [0.566, 1.02]
[400, 3, 32, 0] [0.55, 0.949]
[450, 3, 32, 0] [0.38, 0.745]
[500, 3, 32, 0] [0.391, 0.77]
[10, 3, 32, 1] [1.968, 1.414]
[50, 3, 32, 1] [1.184, 1.013]
[100, 3, 32, 1] [0.434, 0.794]
[150, 3, 32, 1] [0.396, 0.827]
[200, 3, 32, 1] [0.41, 0.849]
[250, 3, 32, 1] [0.362, 0.806]
[300, 3, 32, 1] [0.495, 0.912]
[350, 3, 32, 1] [0.464

[200, 5, 64, 1] [0.523, 0.857]
[250, 5, 64, 1] [0.614, 1.043]
[300, 5, 64, 1] [0.511, 0.87]
[350, 5, 64, 1] [0.463, 0.834]
[400, 5, 64, 1] [0.475, 0.883]
[450, 5, 64, 1] [0.417, 0.985]
[500, 5, 64, 1] [0.502, 0.856]
--------------------------------------------------
Minimimu Error at
[300, 2, 4, 0]
[0.811, 0.08, 0.145, 0.068, 0.077, 0.878, 0.261, 0.224, 0.773, 1.08, 0.236, 0.643]
